# 待解問題
- 時間
- 內文格式不同的狀況

In [1]:
import re
import json
import pandas as pd
import time
from collections import defaultdict
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

In [2]:
browser = webdriver.Chrome()
browser.get('https://www.facebook.com/groups/1189431674567215/')
html = browser.page_source

In [5]:
# 滾動卷軸
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [9]:
# 點擊查看更多
read_button = browser.find_elements_by_css_selector("div[class='oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl oo9gr5id gpro0wi8 lrazzd5p'][role='button']")
for read in read_button:
    time.sleep(1)
    read.click()

WebDriverException: Message: unknown error: Element <div class="oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl oo9gr5id gpro0wi8 lrazzd5p" role="button" tabindex="0">...</div> is not clickable at point (415, 88). Other element would receive the click: <div class="rq0escxv l9j0dhe7 du4w35lb j83agx80 g5gj957u rj1gh0hx buofh1pr hpfvmrgz i1fnvgqd bp9cbjyn owycx6da btwxx1t3 dlv3wnog rl04r1d5">...</div>
  (Session info: chrome=91.0.4472.124)
  (Driver info: chromedriver=2.38.552522 (437e6fbedfa8762dec75e2c5b3ddb86763dc9dcb),platform=Windows NT 10.0.19041 x86_64)


In [173]:
all_cont = browser.find_elements_by_css_selector("div[class='rq0escxv l9j0dhe7 du4w35lb']")

In [174]:
company = defaultdict(dict)

for cont in all_cont:
        sens = cont.text.split('\n')
        for index, info in enumerate(sens):
            
            if re.search(r'的真實價值', info):
                sub_content = re.split(',|、|的真實價值',info)
                for i, sub in enumerate(sub_content):
                    #print(i, sub)
                    code = re.search(r'\d{4}', sub)                        
                    if code:
                        code = code.group()
                        name = re.split(r'\(|（', sub)[0].strip()
                        name = re.sub(r'.\n', '', name)
                        company[name]['Code'] = code
                        
                        if 'timestamp' not in company[name].keys():
                            ts = sens[index-2]
                            company[name]['timestamp'] = ts

            elif re.search(r'的 Ks =', info):
                index = re.split('的 Ks =', info)
                index_name = index[0].strip()
                index_value = index[1].strip()
                company[index_name]['Ks'] = index_value

            elif re.search(r'的 P / E =', info):
                index = re.split('的 P / E =', info)
                index_name = index[0].strip()
                index_value = index[1].split('=')[-1].strip()
                company[index_name]['P / E'] = index_value

            elif re.search(r'的 FRV =', info):
                index = re.split('的 FRV =', info)
                index_name = index[0].strip()
                index_value = index[1].split('=')[-1].strip()
                company[index_name]['FRV'] = index_value

            elif re.search(r'的 泡沫程度 =', info):
                index = re.split('的 泡沫程度 =', info)
                index_name = index[0].strip()
                index_value = index[1].split('=')[-1].strip()
                company[index_name]['泡沫程度'] = index_value

In [175]:
company

defaultdict(dict,
            {'霖宏': {'Code': '5464',
              'timestamp': '6月24日上午1:28',
              'Ks': '0.056',
              'P / E': '17.86',
              'FRV': '32.68',
              '泡沫程度': '0.825'},
             '美琪瑪': {'Code': '4721',
              'timestamp': '18 小時',
              'Ks': '0.046',
              'P / E': '21.74',
              'FRV': '66.09',
              '泡沫程度': '1.099'},
             '安勤': {'Code': '3479',
              'timestamp': '18 小時',
              'Ks': '0.053',
              'P / E': '18.87',
              'FRV': '51.70',
              '泡沫程度': '1.035'},
             '臻鼎': {'Code': '4958',
              'timestamp': '19 小時',
              'Ks': '0.065',
              'P / E': '15.38',
              'FRV': '140.88',
              '泡沫程度': '0.749'},
             '元翎': {'Code': '4564',
              'timestamp': '19 小時',
              'Ks': '0.059',
              'P / E': '16.95',
              'FRV': '48.48',
              '泡沫程度': '0.969'},

In [105]:
# test
for t, ts in zip(titles, timestamp):
    if t.text is not '' and ts.text is not '':
        content = t.text
        time = ts.text
        print(content, time)

霖宏    (  5464  )    的真實價值    : 2020年12月19日
美琪瑪    (  4721  )   的真實價值    : 6月24日上午1:28
安勤   (  3479  )   的真實價值    : 2 小時
臻鼎    (  4958  )   的真實價值    : 12 小時
元翎    (  4564  )    的真實價值    : 16 小時
鼎元    (  2426  )    的真實價值    : 16 小時


In [115]:
for t in titles:
    if t.text is not '':
        print(t.text)

霖宏    (  5464  )    的真實價值    :
美琪瑪    (  4721  )   的真實價值    :
安勤   (  3479  )   的真實價值    :
臻鼎    (  4958  )   的真實價值    :
元翎    (  4564  )    的真實價值    :
鼎元    (  2426  )    的真實價值    :
錩新    (  2415  )    的真實價值    :
皇田    (  9951  )    的真實價值    :
崇越電    (  3388  )   的真實價值    :
大江    (  8436  )    的真實價值    :


In [107]:
for t in timestamp:
    if t.text :
        print(t.text)

2020年12月19日
6月24日上午1:28
2 小時
12 小時
16 小時
16 小時
17 小時
17 小時
17 小時
